### <center> This notebook has the postprocess of raw Level 3 ASCAT/QuikSCAT data and the following construction of a surface wind dataset for the tropical band and the coastal bands of the south east and north east pacific.


The data used will be the full global grid of the ASCAT/QuikSCAT dataset. The idea is to load the data into chunks and then extract only the tropical band and the whole east pacific. With that data on hand a hovmoller will be built from the meridional average of the tropical dataset and the zonal average of the coastal dataset. The variables to be used will be the meridional and zonal componentes of the 10 meter winds (not the stress, since S2S gives only 10m wind forecast). In addition for the coastal bands the alongshore wind will be computed using the land/sea mask. 

In [3]:
# Imports
import xarray as xr
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

# Just for supressing an annoying warning
import dask
dask.config.set({"array.slicing.split_large_chunks": False})

In [4]:
# Load ASCAT/QuikSCAT masks
mask = xr.open_dataset('data/ASCAT_masks.nc')

In [5]:
# Load scatterometer data

# ASCAT
path = [glob(f'data/QuikSCAT/{yr}*') for yr in range(1999,2009+1)]
path = sorted(sum(path,[]))

def preprocess(data):
    print(data.time.dt.strftime('%F').data)
    data = data.rename({'latitude':'lat','longitude':'lon'}).squeeze().drop('depth')
    data = data[['wind_stress',
                 'surface_downward_eastward_stress',
                 'surface_downward_northward_stress',
                 'eastward_wind',
                 'northward_wind',
                 'wind_speed']]
    data = data.rename({'wind_stress':'tau',
                 'surface_downward_eastward_stress':'taux',
                 'surface_downward_northward_stress':'tauy',
                 'eastward_wind':'u',
                 'northward_wind':'v',
                 'wind_speed':'ws'})
    data = data.sel(lat=slice(-45,45))
    data.coords['lon'] = xr.where(data.lon<0,data.lon+360,data.lon)
    data = data.sortby('lon').sel(lon=slice(110,291))
    return data.compute()

scat = xr.open_mfdataset(path, preprocess=preprocess, parallel=True, concat_dim='time', combine='nested')

['2008-11-24']
['2000-01-03']
['2007-12-21']
['2000-09-29']
['2005-01-20']
['2006-08-26']
['2006-11-05']
['2009-09-01']
['2004-01-30']
['2008-11-19']
['2004-06-11']
['2006-03-29']
['2003-01-01']
['2005-10-22']
['2005-01-18']
['2006-01-26']
['2002-04-05']
['2009-03-23']
['2007-11-29']
['2001-10-19']
['2005-01-30']
['2002-10-13']
['2003-02-27']
['2006-01-19']
['2008-02-02']
['2008-01-01']
['2007-02-11']
['2009-05-04']
['2007-12-18']
['2009-06-29']
['2000-10-17']
['2003-05-20']
['2001-02-06']
['2006-01-27']
['2008-03-24']
['2004-04-24']
['2000-10-04']
['2006-10-02']
['2003-05-04']
['2001-10-28']
['2003-11-26']
['2005-02-19']
['2005-07-14']
['2004-03-14']
['2002-09-26']
['2009-10-01']
['2002-05-20']
['2006-09-17']
['2005-11-04']
['2005-11-09']
['2004-10-28']['2002-08-23']
['2000-01-25']
['2007-04-06']
['2007-02-09']['2001-12-31']['2001-10-01']
['2003-10-30']['2001-10-26']

['2009-02-28']



['2000-11-16']
['2003-08-19']
['2005-04-30']
['2005-10-25']
['2009-10-22']
['2006-03-05']
['2002-07-

In [6]:
scat = scat.drop_duplicates('time')
scat

<xarray.Dataset>
Dimensions:  (time: 3637, lat: 360, lon: 724)
Coordinates:
  * time     (time) datetime64[ns] 1999-10-28T12:00:00 ... 2009-11-17T12:00:00
  * lat      (lat) float32 -44.88 -44.62 -44.38 -44.12 ... 44.38 44.62 44.88
  * lon      (lon) float32 110.1 110.4 110.6 110.9 ... 290.1 290.4 290.6 290.9
Data variables:
    tau      (time, lat, lon) float32 0.3378 0.3208 0.3316 ... nan nan nan
    taux     (time, lat, lon) float32 0.3271 0.3099 0.3186 ... nan nan nan
    tauy     (time, lat, lon) float32 0.0845 0.0832 0.0918 ... nan nan nan
    u        (time, lat, lon) float32 11.68 11.42 11.51 11.49 ... nan nan nan
    v        (time, lat, lon) float32 3.02 3.07 3.32 3.49 ... nan nan nan nan
    ws       (time, lat, lon) float32 13.11 12.85 13.01 13.06 ... nan nan nan
Attributes: (12/21)
    Conventions:              CF-1.4
    title:                    Daily QuikSCAT global wind field
    institution:              ifremer
    source:                   produced at 2013-02-04
    bulletin_date:            2013-02-04
    contact:                  fpaf@ifremer.fr, Abderrahim.Bentamy@ifremer.fr,...
    ...                       ...
    grid_resolution:          0.250 degree
    file_quality_index:       0.0
    time_resolution:          daily
    objective_method:         krigeage
    polar_sea_ice_mask_date:  1999-10-28
    creation_date_time:       20130204T102807

---

##### Tropical hovmoller

In [40]:
# Build tropical hovmoller
tropical = scat.sel(lat=slice(-3,3),lon=slice(mask.lon.min(),mask.lon.max()))
tropical = tropical.where(mask.tropicalmask==1).mean('lat')
tropical = tropical.interpolate_na('lon')  # Inteprolate 180°E nans
tropical = tropical.interpolate_na('time') # Fill missing fields with interpolation of inbetween fields
tropical = tropical.dropna('lon') # Drop all land nans
tropical

<xarray.Dataset>
Dimensions:  (time: 3637, lon: 566)
Coordinates:
  * time     (time) datetime64[ns] 1999-10-28T12:00:00 ... 2009-11-17T12:00:00
  * lon      (lon) float32 140.1 140.4 140.6 140.9 ... 280.6 280.9 281.1 281.4
Data variables:
    tau      (time, lon) float32 0.01533 0.01361 0.01616 ... 0.1075 0.0999
    taux     (time, lon) float32 -0.01474 -0.01319 -0.01571 ... 0.0765 0.0751
    tauy     (time, lon) float32 -0.0028 -0.002119 -0.002419 ... 0.0756 0.0659
    u        (time, lon) float32 -2.652 -2.625 -3.055 -3.29 ... 5.437 5.82 6.04
    v        (time, lon) float32 -0.3069 -0.2638 -0.3206 ... 6.407 5.75 5.3
    ws       (time, lon) float32 2.874 2.819 3.174 3.374 ... 8.78 8.73 8.38 8.12

---

##### Build coastal hovmollers

In [41]:
# north
coastnorth = scat.sel(lon=slice(230,295), lat=slice(0,50))
coastnorth = coastnorth.where(mask.coastmask_north==1).mean('lon')
coastnorth = coastnorth.interpolate_na('time').dropna('lat')
coastnorth

<xarray.Dataset>
Dimensions:  (lat: 152, time: 3637)
Coordinates:
  * lat      (lat) float64 2.125 2.375 2.625 2.875 ... 39.12 39.38 39.62 39.88
  * time     (time) datetime64[ns] 1999-10-28T12:00:00 ... 2009-11-17T12:00:00
Data variables:
    tau      (time, lat) float32 0.1107 0.0951 0.09457 ... 0.06775 0.1037
    taux     (time, lat) float32 0.08128 0.07187 0.07045 ... 0.04395 0.06058
    tauy     (time, lat) float32 0.07455 0.0619 0.06265 ... 0.05112 0.08417
    u        (time, lat) float32 6.345 6.11 6.0 5.67 ... 1.92 2.095 2.452 2.915
    v        (time, lat) float32 5.8 5.225 5.318 3.955 ... 2.148 2.16 2.81 4.057
    ws       (time, lat) float32 8.458 7.887 7.89 7.023 ... 6.34 6.375 7.08 8.48

In [42]:
# south
coastsouth = scat.sel(lon=slice(260,295), lat=slice(-50,0))
coastsouth = coastsouth.where(mask.coastmask_south==1).mean('lon')
coastsouth = coastsouth.interpolate_na('time').dropna('lat').sortby('lat', ascending=False)
coastsouth

<xarray.Dataset>
Dimensions:  (lat: 152, time: 3637)
Coordinates:
  * lat      (lat) float64 -2.125 -2.375 -2.625 -2.875 ... -39.38 -39.62 -39.88
  * time     (time) datetime64[ns] 1999-10-28T12:00:00 ... 2009-11-17T12:00:00
Data variables:
    tau      (time, lat) float32 0.03512 0.03715 0.04435 ... 0.0548 0.0589
    taux     (time, lat) float32 0.0226 0.0246 0.03267 ... 0.04037 0.0489 0.0528
    tauy     (time, lat) float32 0.02622 0.0272 0.02942 ... -0.0246 -0.02607
    u        (time, lat) float32 3.088 3.29 4.002 3.877 ... 4.812 5.255 5.457
    v        (time, lat) float32 3.61 3.68 3.602 2.41 ... -2.287 -2.64 -2.693
    ws       (time, lat) float32 5.07 5.233 5.697 5.19 ... 5.615 6.155 6.36

---

##### Add coordinate and metadata

In [ ]:
# Import some common functions from utils.py file
from utils import haversine, distances, coastcoords, add_hovmoller_coordinatedata
ntropical,ncoastnorth,ncoastsouth = add_hovmoller_coordinatedata(tropical,coastnorth,coastsouth,mask,
                                                                 ['time','distance','lat','lon','index','tau','taux','tauy'])
tropical   = ntropical.copy()
coastnorth = ncoastnorth.copy()
coastsouth = ncoastsouth.copy()
del ntropical, ncoastnorth, ncoastsouth

In [49]:
tropical

<xarray.Dataset>
Dimensions:   (time: 3637, distance: 566)
Coordinates:
  * time      (time) datetime64[ns] 1999-10-28T12:00:00 ... 2009-11-17T12:00:00
  * distance  (distance) float64 0.0 27.8 55.6 ... 1.565e+04 1.568e+04 1.571e+04
    lat       (distance) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    lon       (distance) float32 140.1 140.4 140.6 140.9 ... 280.9 281.1 281.4
    index     (distance) int64 0 1 2 3 4 5 6 7 ... 559 560 561 562 563 564 565
Data variables:
    tau       (time, distance) float32 0.01533 0.01361 0.01616 ... 0.1075 0.0999
    taux      (time, distance) float32 -0.01474 -0.01319 ... 0.0765 0.0751
    tauy      (time, distance) float32 -0.0028 -0.002119 ... 0.0756 0.0659

Now the data is ready with the correct coordinates and dimensions: a time dimension and spatial dimensions in relation to the geographic latitude, longitude along with a numerical index and the distance (in km) of the coastnorth/coast pathway. With this done now some attributes and metadata will be added to the dataset 

In [50]:
dummy = xr.open_dataset('data/ASCAT/2007032100_2007032200_daily-ifremer-L3-MWF-GLO-20110531232401-01.0.nc') # dummy dataset with some attributes

# Global attributes
tropical_attrs   = {'name':'Tropical Hovmoller of ASCAT winds',
                    'institution':'Centro de estudios avanzados en zonas aridas (CEAZA)',
                    'description':'ASCAT average along the tropical band (2°S - 2°N)',
                    'author':'Lucas Glasner',
                    'contact':'lucas.glasner@ceaza.cl'}
coastnorth_attrs = {'name':'Coastal Hovmoller of ASCAT winds for the North East Pacific',
                    'institution':'Centro de estudios avanzados en zonas aridas (CEAZA)',
                    'description':'ASCAT average along a coastal band of 1deg latitude',
                    'author':'Lucas Glasner',
                    'contact':'lucas.glasner@ceaza.cl'}
coastsouth_attrs = {'name':'Coastal Hovmoller of ASCAT winds for the South East Pacific',
                    'institution':'Centro de estudios avanzados en zonas aridas (CEAZA)',
                    'description':'ASCAT average along a coastal band of 1deg latitude',
                    'author':'Lucas Glasner',
                    'contact':'lucas.glasner@ceaza.cl'}

# Common variable attributes
tau_attrs        = dummy.wind_stress.attrs
taux_attrs       = dummy.surface_downward_eastward_stress.attrs
tauy_attrs       = dummy.surface_downward_northward_stress.attrs
lon_attrs        = dummy.longitude.attrs
lat_attrs        = dummy.latitude.attrs
time_attrs       = dummy.time.attrs
index_attrs      = {'long_name':'Position along the spatial dimension of the hovmoller grid',
                    'short_name':'index',
                    'units':'(-)'}
distance_attrs   = {'long_name':'Distance along the spatial dimension of the hovmoller grid',
                    'short_name':'distance',
                    'units':'km'}

In [51]:
# Assign attributes 
tropical.attrs          = tropical_attrs
tropical.tau.attrs      = tau_attrs
tropical.taux.attrs      = taux_attrs
tropical.tauy.attrs     = tauy_attrs
tropical.lon.attrs      = lon_attrs
tropical.lat.attrs      = lat_attrs
tropical.index.attrs    = index_attrs
tropical.distance.attrs = distance_attrs

coastnorth.attrs          = coastnorth_attrs
coastnorth.tau.attrs      = tau_attrs
coastnorth.taux.attrs      = taux_attrs
coastnorth.tauy.attrs     = tauy_attrs
coastnorth.lon.attrs      = lon_attrs
coastnorth.lat.attrs      = lat_attrs
coastnorth.index.attrs    = index_attrs
coastnorth.distance.attrs = distance_attrs

coastsouth.attrs          = coastsouth_attrs
coastsouth.tau.attrs      = tau_attrs
coastsouth.taux.attrs      = taux_attrs
coastsouth.tauy.attrs     = tauy_attrs
coastsouth.lon.attrs      = lon_attrs
coastsouth.lat.attrs      = lat_attrs
coastsouth.index.attrs    = index_attrs
coastsouth.distance.attrs = distance_attrs

In [53]:
# Save data into disk
tropical.to_netcdf('./data/HOVMOLLERS/QuikSCAT_hovmoller_tropical_1999-2009.nc')
coastnorth.to_netcdf('./data/HOVMOLLERS/QuikSCAT_hovmoller_coastnorth_1999-2009.nc')
coastsouth.to_netcdf('./data/HOVMOLLERS/QuikSCAT_hovmoller_coastsouth_1999-2009.nc')

Text(0, 0.5, 'Tropical - South America Coast')

In [54]:
# Quicklook to the hovmollers
fig, ax = plt.subplots(2,3, figsize=(18,10), dpi=100, facecolor='w',
                       sharex=True, sharey=True)

xr.concat([tropical,coastnorth], 'distance').tau.plot(ax=ax[0,0], vmin=0, vmax=0.1, cmap='viridis')
xr.concat([tropical,coastnorth], 'distance').taux.plot(ax=ax[0,1], vmin=-0.1,vmax=0.1, cmap='RdBu_r')
xr.concat([tropical,coastnorth], 'distance').tauy.plot(ax=ax[0,2], vmin=-0.1,vmax=0.1, cmap='RdBu_r')

xr.concat([tropical,coastsouth], 'distance').tau.plot(ax=ax[1,0], vmin=0, vmax=0.1, cmap='viridis')
xr.concat([tropical,coastsouth], 'distance').taux.plot(ax=ax[1,1], vmin=-0.1,vmax=0.1, cmap='RdBu_r')
xr.concat([tropical,coastsouth], 'distance').tauy.plot(ax=ax[1,2], vmin=-0.1,vmax=0.1, cmap='RdBu_r')

ax[0,0].set_title('Wind stress (Pa)')
ax[0,1].set_title('Zonal Wind stress (Pa)')
ax[0,2].set_title('Meridional Wind stress (Pa)')

ax[0,0].set_ylabel('Tropical - North America Coast')
ax[1,0].set_ylabel('Tropical - South America Coast')